In [108]:
import re
import numpy as np

with open("data.txt") as file:
    data = file.read()

In [109]:
pipe_map = []
length_map = []

for line in  data.splitlines():
    pipe_map.append(list(line))
    length_map.append([0 for _ in range(len(line))])

pipe_dict = {
    "|": [(1, 0), (-1, 0)],
    "-": [(0, 1), (0, -1)],
    "L": [(-1, 0), (0, 1)],
    "J": [(-1, 0), (0, -1)],
    "7": [(1, 0), (0, -1)],
    "F": [(1, 0), (0, 1)],
}

start_cord = [
    (y, x)
    for y, row in enumerate(pipe_map)
    for x, cell in enumerate(row)
    if cell == "S"
][0]

start_neighbours = [
    (start_cord[0] + 1, start_cord[1]),
    (start_cord[0] - 1, start_cord[1]),
    (start_cord[0], start_cord[1] + 1),
    (start_cord[0], start_cord[1] - 1),
]

possible_neighbours = []

for cord in start_neighbours:
    if cord[0] >= 0 and cord[1] >= 0:
        pipe = pipe_map[cord[0]][cord[1]]
        if pipe != ".":
            pipe_dirs = pipe_dict[pipe]
            possible_cords = [
                (cord[0] + pipe_dirs[0][0], cord[1] + pipe_dirs[0][1]),
                (cord[0] + pipe_dirs[1][0], cord[1] + pipe_dirs[1][1]),
            ]
            if start_cord in possible_cords:
                possible_neighbours.append(cord)


def generate_lenghts(last_position, current_position):

    count_map = length_map.copy()
    steps = 1
    path = []

    while current_position != start_cord:

        count_map[current_position[0]][current_position[1]] = steps

        pipe = pipe_map[current_position[0]][current_position[1]]
        #print(current_position, pipe)
        
        pipe_dirs = pipe_dict[pipe]
        possible_cords = [
            (current_position[0] + pipe_dirs[0][0], current_position[1] + pipe_dirs[0][1]),
            (current_position[0] + pipe_dirs[1][0], current_position[1] + pipe_dirs[1][1]),
        ]

        possible_cords.remove(last_position)
        path.append(last_position)
        last_position = current_position
        current_position = possible_cords[0]
        steps += 1
    
    path.append(last_position)
    return np.array(count_map), path

path_1, p1 = generate_lenghts(start_cord, possible_neighbours[0])
path_2, p2 = generate_lenghts(start_cord, possible_neighbours[1])

path_1[(path_2 == path_1) & (path_1 != 0) & (path_2 != 0)]

array([6931])

In [126]:
inside = set()
outside = set()

last_cord = p2[-1]
path_set = set(p2)

all_cords = {(y, x) for y, row in enumerate(pipe_map) for x, _ in enumerate(row)}

left_dir = {
    (1, 0): (0, 1),
    (-1, 0): (0, -1),
    (0, 1): (-1, 0),
    (0, -1): (1, 0),
}

right_dir = {
    (1, 0): (0, -1),
    (-1, 0): (0, 1),
    (0, 1): (1, 0),
    (0, -1): (-1, 0),
}

def add_neighbours(point, current_set):

    current_set.add(point)

    neighbours = [
        (point[0] + 1, point[1]),
        (point[0] - 1, point[1]),
        (point[0], point[1] + 1),
        (point[0], point[1] - 1),
    ]

    neighbours = [
        n
        for n in neighbours
        if n in all_cords
    ]

    for neigbour in neighbours:
        if (neigbour not in current_set) and (neigbour not in path_set):
            add_neighbours(neigbour, current_set)


for current_cord in p2:

    heading = np.array(current_cord) - np.array(last_cord)
    r_dir = right_dir[tuple(heading)]
    l_dir = left_dir[tuple(heading)]

    if (
        (left_y := l_dir[0] + current_cord[0]),
        (left_x := l_dir[1] + current_cord[1]),
    ) in all_cords:
        if (left_y, left_x) not in path_set:
            add_neighbours((left_y, left_x), inside)

    if (
        (rigth_y := r_dir[0] + current_cord[0]),
        (right_x := r_dir[1] + current_cord[1]),
    ) in all_cords:
        if (rigth_y, right_x) not in path_set:
            add_neighbours((rigth_y, right_x), outside)

    last_cord = current_cord

In [134]:
len(outside)

357